In [347]:
import pandas as pd
import numpy as np
import os
import re
import pandas_datareader.data as web

from pathlib import Path
from IPython.display import display
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [348]:
# Remove all data without at least 5 years worth of data
def set_length(data, year):
    counts = data.groupby('permno').size()
    to_exclude = counts[counts < (year * 12)].index
    mask = ~data.index.get_level_values('permno').isin(to_exclude)
    data = data[mask]
    return data

# Get ticker data
def get_permno_data(data, permno):
    idx = pd.IndexSlice
    desired_data = data.loc[idx[:, permno, :], :]
    return desired_data

# Get ticker data
def get_ind_data(data, ind):
    idx = pd.IndexSlice
    desired_data = data.loc[idx[ind, :, :], :]
    return desired_data

# Assign industry based off range
def assign_label(df, column_name, sic_ranges, label):
    df['sic_temp_crsp'] = df['sic_crsp']
    df['sic_temp_comp'] = df['sic_comp']
    
    for r in sic_ranges:
        if isinstance(r, tuple):
            df.loc[(df['sic_temp_crsp'] >= r[0]) & (df['sic_temp_crsp'] <= r[1]), f'{column_name}_crsp'] = label
            df.loc[(df['sic_temp_comp'] >= r[0]) & (df['sic_temp_comp'] <= r[1]), f'{column_name}_comp'] = label
        else:
            df.loc[df['sic_temp_crsp'] == r, f'{column_name}_crsp'] = label
            df.loc[df['sic_temp_comp'] == r, f'{column_name}_comp'] = label
    
    df = df.drop(columns=['sic_temp_crsp', 'sic_temp_comp'])
    
    return df

In [349]:
root = Path(os.getcwd())

In [350]:
comp = pd.read_csv(root / 'data' / 'comp_price.csv')
crsp_comp = pd.read_csv(root / 'data' / 'crsp_comp_price.csv')
crsp = pd.read_csv(root / 'data' / 'crsp_price.csv')

In [351]:
comp['datadate'] = pd.to_datetime(comp['datadate']).dt.to_period("M").dt.to_timestamp("M")
comp = comp.rename(columns={'tic': 'ticker', 'prccm':'close', 'cshom': 'out_share', 'datadate': 'date'}
                              ).set_index(['ticker', 'date']).drop(['iid', 'gvkey'], axis=1)

In [352]:
crsp_comp['datadate'] = pd.to_datetime(crsp_comp['datadate']).dt.to_period("M").dt.to_timestamp("M")
crsp_comp = crsp_comp.rename(columns={'tic': 'ticker', 'prccm':'close', 'cshoq': 'out_share', 'datadate': 'date'
                                     , 'LPERMNO':'permno', 'sic':'sic_comp'}
                              ).set_index(['permno', 'date']).drop(['iid', 'GVKEY', 'ticker', 'out_share', 'close'], axis=1)
crsp_comp = crsp_comp.sort_index(level=['permno','date'])

In [353]:
crsp['date'] = pd.to_datetime(crsp['date']).dt.to_period("M").dt.to_timestamp("M")

crsp = crsp.rename(columns={'TICKER': 'ticker', 'PRC':'close', 'VOL':'volume', 
                                        'SHROUT': 'out_share', 'FACSHR': 'adj_share', 'ISSUNO':'nasdaq',
                                        'SHRCD': 'share_code', 'PERMNO':'permno', 'SICCD':'sic_crsp', 'RET':'ret_01'}
                              ).set_index(['permno', 'date']).drop('PERMCO', axis=1)

crsp = crsp.sort_index(level=['permno', 'date'])

crsp = crsp[crsp.index.get_level_values('permno').notna()]

crsp['sic_crsp'] = pd.to_numeric(crsp['sic_crsp'], errors='coerce', downcast='integer')

crsp = crsp.dropna(subset=['sic_crsp'])

In [354]:
combined = pd.merge(crsp, crsp_comp, left_index=True, right_index=True, how='outer')
combined = combined[~combined.index.duplicated(keep='first')]

In [355]:
# Make a permno's industry the same across all dates
# def first_non_na(series):
#     return series.dropna().iloc[0] if not series.dropna().empty else np.nan

# combined['sic_crsp'] = combined.groupby('permno')['sic_crsp'].transform(first_non_na)
# combined['sic_comp'] = combined.groupby('permno')['sic_comp'].transform(first_non_na)

In [356]:
# combined = set_length(combined, 2)

# Create Ind

In [357]:
ind = {
    'agric': [(100, 199), (200, 299), (700, 799), (910, 919), 2048],
    'food': [(2000, 2009), (2010, 2019), (2020, 2029), (2030, 2039), (2040, 2046), (2050, 2059), (2060, 2063), (2070, 2079), (2090, 2092), 2095, (2098, 2099)],
    'soda': [(2064, 2068), 2086, 2087, 2096, 2097],
    'beer': [2080, 2082, 2083, 2084, 2085],
    'smoke': [(2100, 2199)],
    'toys': [(920, 999), (3650, 3651), 3652, 3732, (3930, 3931), (3940, 3949)],
    'fun': [(7800, 7829), (7830, 7833), (7840, 7841), 7900, (7910, 7911), (7920, 7929), (7930, 7933), (7940, 7949), 7980, (7990, 7999)],
    'books': [(2700, 2709), (2710, 2719), (2720, 2729), (2730, 2739), (2740, 2749), (2770, 2771), (2780, 2789), (2790, 2799)],
    'hshld': [2047, (2391, 2392), (2510, 2519), (2590, 2599), (2840, 2843), 2844, (3160, 3161), (3170, 3171), 3172, (3190, 3199), 3229, 3260, (3262, 3263), 3269, (3230, 3231), (3630, 3639), (3750, 3751), 3800, (3860, 3861), (3870, 3873), (3910, 3911), 3914, 3915, (3960, 3962), 3991, 3995],
    'clths': [(2300, 2390), (3020, 3021), (3100, 3111), (3130, 3131), (3140, 3149), (3150, 3151), (3963, 3965)],
    'hlth': [(8000, 8099)],
    'medeq': [3693, (3840, 3849), (3850, 3851)],
    'drugs': [2830, 2831, 2833, 2834, 2835, 2836],
    'chems': [(2800, 2809), (2810, 2819), (2820, 2829), (2850, 2859), (2860, 2869), (2870, 2879), (2890, 2899)],
    'rubbr': [3031, 3041, (3050, 3053), (3060, 3069), (3070, 3079), (3080, 3089), (3090, 3099)],
    'txtls': [(2200, 2269), (2270, 2279), (2280, 2284), (2290, 2295), 2297, 2298, 2299, (2393, 2395), (2397, 2399)],
    'bldmt': [(800, 899), (2400, 2439), (2450, 2459), (2490, 2499), (2660, 2661), (2950, 2952), 3200, (3210, 3211), (3240, 3241), (3250, 3259), 3261, 3264, (3270, 3275), (3280, 3281), (3290, 3293), (3295, 3299), (3420, 3429), (3430, 3433), (3440, 3441), 3442, 3446, 3448, 3449, (3450, 3451), 3452, (3490, 3499), 3996],
    'cnstr': [(1500, 1511), (1520, 1529), (1530, 1539), (1540, 1549), (1600, 1699), (1700, 1799)],
    'steel': [3300, (3310, 3317), (3320, 3325), (3330, 3339), (3340, 3341), (3350, 3357), (3360, 3369), (3370, 3379), (3390, 3399)],
    'fabpr': [3400, 3443, 3444, (3460, 3469), (3470, 3479)],
    'mach': [(3510, 3519), (3520, 3529), 3530, 3531, 3532, 3533, 3534, 3535, 3536, 3538, (3540, 3549), (3550, 3559), (3560, 3569), 3580, 3581, 3582, 3585, 3586, 3589, (3590, 3599)],
    'elceq': [3600, (3610, 3613), (3620, 3621), (3623, 3629), (3640, 3644), 3645, 3646, (3648, 3649), 3660, 3690, (3691, 3692), 3699],
    'autos': [2296, 2396, (3010, 3011), 3537, 3647, 3694, 3700, 3710, 3711, 3713, 3714, 3715, 3716, 3792, (3790, 3791), 3799],
    'aero': [3720, 3721, (3723, 3724), 3725, (3728, 3729)],
    'ships': [(3730, 3731), (3740, 3743)],
    'guns': [(3760, 3769), 3795, (3480, 3489)],
    'gold': [(1040, 1049)],
    'mines': [(1000, 1009), (1010, 1019), (1020, 1029), (1030, 1039), (1050, 1059), (1060, 1069), (1070, 1079), (1080, 1089), (1090, 1099), (1100, 1119), (1400, 1499)],
    'coal': [(1200, 1299)],
    'oil': [1300, (1310, 1319), (1320, 1329), (1330, 1339), (1370, 1379), 1380, 1381, 1382, 1389, (2900, 2912), (2990, 2999)],
    'util': [4900, (4910, 4911), (4920, 4922), 4923, (4924, 4925), (4930, 4931), 4932, 4939, (4940, 4942)],
    'telcm': [4800, (4810, 4813), (4820, 4822), (4830, 4839), (4840, 4841), 4880, 4890, 4891, 4892, 4899],
    'persv': [(7020, 7021), (7030, 7033), 7200, (7210, 7212), 7214, (7215, 7216), 7217, 7219, (7220, 7221), (7230, 7231), (7240, 7241), (7250, 7251), (7260, 7269), (7270, 7290), 7291, (7292, 7299), 7395, 7500, (7520, 7529), (7530, 7539), (7540, 7549), 7600, 7620, 7622, 7623, 7629, 7630, 7640, (7690, 7699), (8100, 8199), (8200, 8299), (8300, 8399), (8400, 8499), (8600, 8699), (8800, 8899), (7510, 7515)],
    'bussv': [(2750, 2759), 3993, 7218, 7300, (7310, 7319), (7320, 7329), (7330, 7339), (7340, 7342), 7349, (7350, 7351), 7352, 7353, 7359, (7360, 7369), 7374, 7376, 7377, 7378, 7379, 7380, (7381, 7382), 7383, 7384, 7385, 7389, 7390, 7391, (7392, 7392), 7393, 7394, 7396, 7397, 7399, (7519, 7519), 8700, (8710, 8713), (8720, 8721), (8730, 8734), (8740, 8748), (8900, 8910), 8911, (8920, 8999), (4220, 4229)],
    'hardw': [(3570, 3579), 3680, 3681, 3682, 3683, 3684, 3685, 3686, 3687, 3688, 3689, 3695],
    'softw': [(7370, 7372), 7375, 7373],
    'chips': [3622, 3661, (3662, 3662), 3663, 3664, 3665, 3666, 3669, (3670, 3679), (3810, 3810), (3812, 3812)],
    'labeq': [3811, 3820, 3821, 3822, 3823, 3824, 3825, 3826, 3827, 3829, (3830, 3839)],
    'paper': [(2520, 2549), (2600, 2639), (2670, 2699), (2760, 2761), (3950, 3955)],
    'boxes': [(2440, 2449), (2640, 2659), (3220, 3221), (3410, 3412)],
    'trans': [(4000, 4013), (4040, 4049), 4100, (4110, 4119), (4120, 4121), (4130, 4131), (4140, 4142), (4150, 4151), (4170, 4173), (4190, 4199), 4200, (4210, 4219), (4230, 4231), (4240, 4249), (4400, 4499), (4500, 4599), (4600, 4699), 4700, (4710, 4712), (4720, 4729), (4730, 4739), (4740, 4749), 4780, 4782, 4783, 4784, 4785, 4789],
    'whlsl': [5000, (5010, 5015), (5020, 5023), (5030, 5039), (5040, 5042), 5043, 5044, 5045, 5046, 5047, 5048, 5049, (5050, 5059), 5060, 5063, 5064, 5065, (5070, 5078), 5080, 5081, 5082, 5083, 5084, 5085, (5086, 5087), 5088, 5090, (5091, 5092), 5093, 5094, 5099, 5100, (5110, 5113), (5120, 5122), (5130, 5139), (5140, 5149), (5150, 5159), (5160, 5169), (5170, 5172), (5180, 5182), (5190, 5199)],
    'rtail': [5200, (5210, 5219), (5220, 5229), (5230, 5231), (5250, 5251), (5260, 5261), (5270, 5271), 5300, 5310, 5320, (5330, 5331), 5334, (5340, 5349), (5390, 5399), 5400, (5410, 5411), 5412, (5420, 5429), (5430, 5439), (5440, 5449), (5450, 5459), (5460, 5469), (5490, 5499), 5500, (5510, 5529), (5530, 5539), (5540, 5549), (5550, 5559), (5560, 5569), (5570, 5579), (5590, 5599), (5600, 5699), 5700, (5710, 5719), (5720, 5722), (5730, 5733), 5734, 5735, 5736, (5750, 5799), 5900, (5910, 5912), (5920, 5929), (5930, 5932), 5940, 5941, 5942, 5943, 5944, 5945, 5946, 5947, 5948, 5949, (5950, 5959), (5960, 5969), (5970, 5979), (5980, 5989), 5990, 5992, 5993, 5994, 5995, 5999],
    'meals': [(5800, 5819), (5820, 5829), (5890, 5899), 7000, (7010, 7019), (7040, 7049), 7213],
    'banks': [6000, (6010, 6019), 6020, 6021, 6022, 6023, 6025, 6026, 6027, (6028, 6029), (6030, 6036), (6040, 6059), (6060, 6062), (6080, 6082), (6090, 6099), 6100, (6110, 6111), (6112, 6113), (6120, 6129), (6130, 6139), (6140, 6149), (6150, 6159), (6160, 6169), (6170, 6179), (6190, 6199)],
    'insur': [6300, (6310, 6319), (6320, 6329), (6330, 6331), (6350, 6351), (6360, 6361), (6370, 6379), (6390, 6399), (6400, 6411)],
    'rlest': [6500, 6510, 6512, 6513, 6514, 6515, (6517, 6519), (6520, 6529), (6530, 6531), 6532, (6540, 6541), (6550, 6553), (6590, 6599), (6610, 6611)],
    'fin': [(6200, 6299), 6700, (6710, 6719), (6720, 6722), 6723, 6724, 6725, 6726, (6730, 6733), (6740, 6779), 6790, 6791, 6792, 6793, 6794, 6795, 6798, 6799],
    'other': [(4950, 4959), (4960, 4961), (4970, 4971), (4990, 4991)]
}

In [358]:
for name, ranges in ind.items():
    print('-'*60)
    print(name)
    combined = assign_label(combined, 'ind', ranges, name)

------------------------------------------------------------
agric
------------------------------------------------------------
food
------------------------------------------------------------
soda
------------------------------------------------------------
beer
------------------------------------------------------------
smoke
------------------------------------------------------------
toys
------------------------------------------------------------
fun
------------------------------------------------------------
books
------------------------------------------------------------
hshld
------------------------------------------------------------
clths
------------------------------------------------------------
hlth
------------------------------------------------------------
medeq
------------------------------------------------------------
drugs
------------------------------------------------------------
chems
------------------------------------------------------------
rubbr
--

In [359]:
combined['ind'] = combined['ind_comp'].combine_first(combined['ind_crsp'])

# Ind Data

In [360]:
ind_data = combined.dropna(subset=['ind'])
ind_data = ind_data.reset_index()
ind_data = ind_data.set_index(['ind', 'permno', 'date'])
store = ind_data.copy(deep=True)

In [361]:
# # Remove all industries with no more than 10 firms
# ind_count = ind_data.groupby('permno')['ind'].first().value_counts()
# valid_ind = ind_count[ind_count >= 10].index.tolist()
# ind_data = ind_data[ind_data['ind'].isin(valid_ind)]

### Clean

In [362]:
ind_data = store
ind_data['ret_01'] = pd.to_numeric(ind_data['ret_01'], errors='coerce')
# Drop all permno/date pairs that have negative closing price data
ind_data = ind_data[ind_data['close'] >= 0]
ind_data = ind_data.sort_index(level = ['ind', 'permno', 'date'])

# Acceleration

### Compound

In [363]:
# def calculate_acceleration(data):
#     data = data.reset_index(level=['ind', 'permno'], drop=True)
#     data['1t24'] = (data['ret_01'] + 1).rolling(24).apply(np.prod, raw=True) - 1
#     data['12t24'] = (data['ret_01'] + 1).shift(12).rolling(12).apply(np.prod, raw=True) - 1
#     data['acc'] = data['1t24'] - data['12t24']
#     return data

# ind_data = ind_data.groupby(level=['ind', 'permno']).apply(calculate_acceleration)
# ind_data = ind_data.sort_index(level=['permno', 'date'])

### Pct Change

In [364]:
def calculate_acceleration(group):
    group['1t24'] = group['close'].pct_change(periods=24)
    group['12t24'] = group['close'].pct_change(periods=12).shift(12)
    group['acc'] = group['1t24'] - group['12t24']
    return group

ind_data = ind_data.groupby(level=['ind', 'permno']).apply(calculate_acceleration).reset_index(level=[0, 1], drop=True)
ind_data = ind_data.sort_index(level=['ind', 'permno', 'date'])

# Value-Weighted Returns

In [365]:
ind_data['value_permno'] = ind_data['close'] * ind_data['out_share'] * 1000
ind_data['value_ind'] = ind_data.groupby(['ind', 'date'])['value_permno'].transform('sum')
ind_data['vwr_weight'] = ind_data['value_permno'] / ind_data['value_ind']
ind_data['vwr_permno'] = ind_data['vwr_weight'] * ind_data['ret_01']
ind_data['vwr_ind'] = ind_data.groupby(['ind', 'date'])['vwr_permno'].transform('sum')

# Volatility

In [366]:
daily = pd.read_csv(root / 'data' / 'daily_price.csv')
daily = daily.rename(columns={'PERMNO':'permno', 'PRC':'close'})
daily = daily[daily['close'] >= 0]
daily['ret_01'] = daily.groupby('permno')['close'].pct_change()
daily['date'] = pd.to_datetime(daily['date'])
daily['month'] = daily['date'].dt.to_period('M')

In [367]:
month_vol = daily.groupby(['permno', 'month'])['ret_01'].std().reset_index()
month_vol = month_vol.rename(columns={'month':'date', 'ret_01':'vol'})
month_vol['date'] = month_vol['date'].astype(str)
month_vol['date'] = pd.to_datetime(month_vol['date'])
month_vol['date'] = month_vol['date'].dt.to_period('M').dt.to_timestamp('M')
month_vol = month_vol.set_index(['permno', 'date'])

In [368]:
ind_data = ind_data.reset_index().set_index(['permno', 'date'])
ind_data = pd.merge(ind_data, month_vol, left_index=True, right_index=True, how='left')
ind_data = ind_data.reset_index().set_index(['ind', 'permno', 'date'])
ind_data['vol_ind_rank'] = ind_data.groupby('ind')['vol'].rank(pct=True)
ind_data['vol_value'] = ind_data['vol_ind_rank'] * ind_data['value_permno']
ind_data['vol_ind'] = ind_data.groupby(['ind', 'date'])['vol_value'].transform('sum') / ind_data.groupby(['ind', 'date'])['value_permno'].transform('sum')
ind_data['vol_market_rank'] = ind_data.groupby('date')['vol_ind'].rank(pct=True)
ind_data['vol_weight'] = ind_data['vol'] * ind_data['value_permno']
ind_data['vol_vw'] = ind_data.groupby(['ind', 'date'])['vol_weight'].transform('sum') / ind_data.groupby(['ind', 'date'])['value_permno'].transform('sum')
ind_data['vol_vw_change'] = ind_data.groupby('ind')['vol_vw'].transform(lambda x: (x - x.shift(12)) / x.shift(12))

# Turnover

In [369]:
outstanding = np.where(ind_data['nasdaq'] != 0, 2 * ind_data['out_share'], ind_data['out_share'])
ind_data['turnover'] = (ind_data['volume'] * 100) / (outstanding * 1000)
ind_data['turnover_ind_rank'] = ind_data.groupby('ind')['turnover'].rank(pct=True)
ind_data['turnover_value'] = ind_data['turnover'] * ind_data['value_permno']
ind_data['turnover_vw'] = ind_data.groupby(['ind', 'date'])['turnover_value'].transform('sum') / ind_data.groupby(['ind', 'date'])['value_permno'].transform('sum')
ind_data['turnover_market_rank'] = ind_data.groupby('date')['turnover_vw'].rank(pct=True)
ind_data['turnover_vw_change'] = ind_data.groupby('ind')['turnover_vw'].transform(lambda x: (x - x.shift(12)) / x.shift(12))

# Age

In [370]:
first_date = ind_data.reset_index().groupby('permno')['date'].min()
ind_data['first_date'] = ind_data.index.get_level_values('permno').map(first_date)
ind_data_date = ind_data.index.get_level_values('date')
ind_data['age'] = ((ind_data_date.year - ind_data['first_date'].dt.year) * 12 + ind_data_date.month - ind_data['first_date'].dt.month) // 12
ind_data['age_ind_rank'] = ind_data.groupby('date')['age'].rank(pct=True)
ind_data['age_value'] = ind_data['age_ind_rank'] * ind_data['value_permno']
ind_data['age_ind'] = ind_data.groupby(['ind', 'date'])['age_value'].transform('sum') / ind_data.groupby(['ind', 'date'])['value_permno'].transform('sum')
ind_data['age_market_rank'] = ind_data.groupby('date')['age_ind'].rank(pct=True)

# Age Tilt

In [371]:
ind_data['ewr_ind'] = ind_data.groupby(['ind', 'date'])['ret_01'].transform('mean')
ind_data['age_weight'] = ind_data.groupby(['ind', 'date'])['age'].transform(lambda x: x / x.sum())
ind_data['awr_permno'] = ind_data['ret_01'] * ind_data['age_weight']
ind_data['awr_ind'] = ind_data.groupby(['ind', 'date'])['awr_permno'].transform('sum')
ind_data['age_tilt'] = ind_data['awr_ind'] - ind_data['ewr_ind']

# Sanity Check

In [372]:
# def check(ind, firm):
#     x = ind.loc['1926-07-31':'2022-12-31']
#     y = firm.loc['1926-07-31':'2022-12-31']
#     display(x.head(5))
#     display(y.head(5))

#     collect = []
#     for i in x.columns:
#         print('-'*60)
#         print(i)
#         corr = x[i].corr(y[i])
#         collect.append(corr)
#         print(corr)
#     print('-'*60)
#     print(f'Total Mean: {sum(collect) / len(collect)}')

### Check Firm Size

In [373]:
# fama_firm = pd.read_csv(root / 'data' / 'fama_firm_49.csv')

# fama_firm['date'] = pd.to_datetime(fama_firm['date'].astype(str), format='%Y%m').dt.to_period("M").dt.to_timestamp("M")
# fama_firm.columns = [c.strip() for c in fama_firm.columns]

# date = fama_firm['date']
# fama_firm = fama_firm.drop(columns=['date'])
# fama_firm.columns = [col.lower() for col in fama_firm.columns]
# # fama_firm = fama_firm.drop(columns=['other'])
# fama_firm = fama_firm.sort_index(axis=1)
# fama_firm = pd.concat([date, fama_firm], axis=1).set_index('date')

In [374]:
# temp_df = ind_data.reset_index()
# ind_firm = temp_df.groupby(['date', 'ind']).permno.nunique().unstack()
# ind_firm = ind_firm.sort_index(axis=1)
# ind_firm = ind_firm.fillna(0)

In [375]:
# check(ind_firm, fama_firm)

### Check VWR

In [376]:
# fama_vwr = pd.read_csv(root / 'data' / 'fama_vwr_49.csv')

# fama_vwr['date'] = pd.to_datetime(fama_vwr['date'].astype(str), format='%Y%m').dt.to_period("M").dt.to_timestamp("M")
# fama_vwr.columns = [c.strip() for c in fama_vwr.columns]

# date = fama_vwr['date']
# fama_vwr = fama_vwr.drop(columns=['date'])
# fama_vwr.columns = [col.lower() for col in fama_vwr.columns]
# # fama_vwr = fama_vwr.drop(columns=['other'])
# fama_vwr = fama_vwr.sort_index(axis=1)
# fama_vwr = pd.concat([date, fama_vwr], axis=1).set_index('date')
# fama_vwr = fama_vwr/100

In [377]:
# collect = []
# for industry, df in ind_data.groupby('ind'):
#     df = df[['vwr_ind']]
#     x = df.unstack('permno')
#     x.columns = x.columns.get_level_values(0)
#     x = x.reset_index().drop('ind', axis=1).set_index('date')
#     x[industry] = x.apply(lambda row: row.dropna().iloc[0] if not row.dropna().empty else np.nan, axis=1)
#     collect.append(x[[industry]])

# ind_vwr = pd.concat(collect, axis=1)

In [378]:
# check(ind_vwr, fama_vwr)

### Check EWR Size

In [379]:
# fama_ewr = pd.read_csv(root / 'data' / 'fama_ewr_49.csv')

# fama_ewr['date'] = pd.to_datetime(fama_ewr['date'].astype(str), format='%Y%m').dt.to_period("M").dt.to_timestamp("M")
# fama_ewr.columns = [c.strip() for c in fama_ewr.columns]

# date = fama_ewr['date']
# fama_ewr = fama_ewr.drop(columns=['date'])
# fama_ewr.columns = [col.lower() for col in fama_ewr.columns]
# # fama_ewr = fama_ewr.drop(columns=['other'])
# fama_ewr = fama_ewr.sort_index(axis=1)
# fama_ewr = pd.concat([date, fama_ewr], axis=1).set_index('date')
# fama_ewr = fama_ewr.replace(-99.99, np.NaN)
# fama_ewr = fama_ewr/100

In [380]:
# collect = []
# for industry, df in ind_data.groupby('ind'):
#     df = df[['ewr_ind']]
#     x = df.unstack('permno')
#     x.columns = x.columns.get_level_values(0)
#     x = x.reset_index().drop('ind', axis=1).set_index('date')
#     x[industry] = x.apply(lambda row: row.dropna().iloc[0] if not row.dropna().empty else np.nan, axis=1)
#     collect.append(x[[industry]])
# ind_ewr = pd.concat(collect, axis=1)

In [381]:
# check(ind_ewr, fama_ewr)

# Issuance

In [382]:
ind_data['adj_share_pct'] = ind_data.groupby('permno')['adj_share'].pct_change(periods=12)
ind_data['issued_equity'] = (ind_data['adj_share_pct'] >= 0.05).astype(int)
ind_data['issuance'] = ind_data.groupby(['ind', 'date'])['issued_equity'].transform('mean')

# Book to Market

### Fama

In [383]:
with open(root / 'data' / 'fama_be.txt', 'r') as file:
    lines = file.readlines()
    
full_text = ''.join(lines)
records = re.split(r'(\d{5} )', full_text)[1:]
records = [records[i] + records[i + 1] for i in range(0, len(records), 2)]
data = []

for record in records:
    data.append(record.split())
be_pre  = pd.DataFrame(data)

In [384]:
all_rows = []

for _, row in be_pre.iterrows():
    permno = row[0]
    start_date = int(row[1])
    end_date = int(row[2])
    for year in range(start_date, end_date + 1):
        new_row = row.copy()
        new_row[1] = year
        all_rows.append(new_row)

be_pre = pd.DataFrame(all_rows)

# Set the multi-index
be_pre = be_pre.set_index([0, 1])
be_pre.index.names = ['permno', 'date']
be_pre = be_pre.drop(2, axis=1)

In [385]:
collect = []
for _, df in be_pre.groupby('permno'):
    df['btm']=df.values[0][:len(df)]
    collect.append(df[['btm']])

be_pre = pd.concat(collect, axis = 0)
be_pre = be_pre.reset_index().set_index('date')
be_pre['permno'] = be_pre['permno'].astype(int)

In [386]:
be_pre['btm'] = be_pre['btm'].astype(float)
be_pre['btm'] = be_pre['btm']/100
be_pre = be_pre.sort_index()
be_pre.loc[be_pre.btm == be_pre.btm.iloc[-1], 'btm'] = np.nan
be_pre = be_pre.loc['1926':'1960']
be_pre = be_pre.dropna()

### CRSP

In [387]:
be_curr = pd.read_csv(root / 'data' / 'curr_be.csv')
be_curr['btm'] = (be_curr['atq']-be_curr['ltq'])/(be_curr['prccq']*be_curr['cshoq'])
be_curr = be_curr.rename(columns={'LPERMNO':'permno', 'datadate':'date'})
be_curr['date'] = pd.to_datetime(be_curr['date'])
be_curr = be_curr.set_index('date')
be_curr = be_curr[['permno', 'btm']]

### Combined

In [388]:
def convert_date(x):
    if len(str(x)) == 4:
        return pd.to_datetime(x, format='%Y')
    else:
        return pd.to_datetime(x)

In [389]:
be_total = pd.concat([be_pre, be_curr], axis=0)
be_total = be_total.reset_index('date').set_index(['permno', 'date'])
be_total = be_total.sort_index(level=['permno', 'date'])
btm = be_total.reset_index()

In [390]:
btm['date'] = btm['date'].apply(convert_date)
btm = btm.drop_duplicates(subset=['permno', 'date'], keep='first')
btm = btm.set_index('date')
btm = btm.groupby('permno').apply(lambda group: group.resample('M').ffill())
btm = btm.drop('permno', axis=1)

### Add to ind_data

In [391]:
ind_data = ind_data.reset_index().set_index(['permno', 'date'])
ind_data = pd.merge(ind_data, btm, left_index=True, right_index=True, how='left')
ind_data = ind_data.reset_index()
ind_data['ind'] = ind_data.groupby('permno')['ind'].ffill()
ind_data = ind_data.set_index(['ind', 'permno', 'date'])
ind_data['btm'] = ind_data['btm'].astype(float)
ind_data['btm_weight'] = ind_data['btm'] * ind_data['value_permno']
ind_data['btm_vwr'] = ind_data.groupby(['ind', 'date'])['btm_weight'].transform('sum') / ind_data.groupby(['ind', 'date'])['value_permno'].transform('sum')

# Sales Growth

In [392]:
revenue = pd.read_csv(root / 'data' / 'revenue.csv')
revenue = revenue.rename(columns={'LPERMNO':'permno', 'datadate':'date', 'revt':'revenue'})
revenue = revenue[['permno', 'date', 'revenue']]
revenue['date'] = pd.to_datetime(revenue['date'])
revenue = revenue.sort_values(by=['permno','date'])
revenue['revenue'] = revenue.groupby('permno')['revenue'].pct_change()
revenue = revenue.set_index('date')
revenue = revenue.groupby('permno').apply(lambda group: group.resample('M').ffill())
revenue = revenue.drop('permno', axis=1)

In [393]:
ind_data = ind_data.reset_index().set_index(['permno', 'date'])
ind_data = pd.merge(ind_data, revenue, left_index=True, right_index=True, how='left')
ind_data = ind_data.reset_index().set_index(['ind', 'permno', 'date'])
ind_data['revenue_ind_rank'] = ind_data.groupby('ind')['revenue'].rank(pct=True)
ind_data['revenue_value'] = ind_data['revenue_ind_rank'] * ind_data['value_permno']
ind_data['revenue_ind'] = ind_data.groupby(['ind', 'date'])['revenue_value'].transform('sum') / ind_data.groupby(['ind', 'date'])['value_permno'].transform('sum')
ind_data['revenue_market_rank'] = ind_data.groupby('date')['revenue_ind'].rank(pct=True)

# CAPE

In [394]:
cape = pd.read_csv(root / 'data' / 'cape.csv')
series = cape.date
years = series.astype(int)
months = ((series - years) * 100).round().astype(int)
datetimes = pd.to_datetime(years.astype(str) + '-' + months.astype(str) + '-01')
cape['date'] = datetimes
cape['date'] = cape['date'].dt.to_period("M").dt.to_timestamp("M")
ind_data = ind_data.reset_index().merge(cape.reset_index(), on='date', how='left').set_index(['ind', 'permno', 'date'])

# Total Market Return

In [395]:
ind_data['permno_weighted'] = ind_data['value_permno'] * ind_data['ret_01']
total_weighted_return = ind_data.groupby('date')['permno_weighted'].transform('sum')
total_market_value = ind_data.groupby('date')['value_permno'].transform('sum')
ind_data['market_return'] = total_weighted_return / total_market_value

# 2 Year Return

In [396]:
ind_data['two_vwr_ind'] = ind_data.groupby('ind')['vwr_ind'].rolling(window=24).apply(lambda x: (x + 1).prod() - 1, raw=True).values

# Risk Free Rate

In [397]:
rf = pd.read_csv(root / 'data' / 'risk_free.csv')
rf = rf.drop('KYTREASNOX', axis=1)
rf.columns = ['date', 'rf']
rf['date'] = pd.to_datetime(rf['date'])
rf = rf.set_index('date')
rf = rf.loc[~rf.index.duplicated(keep='first')]
rf = rf.dropna()

# Retrieving paper predictors

In [398]:
ind_data.columns

Index(['share_code', 'sic_crsp', 'ticker', 'nasdaq', 'adj_share', 'close',
       'volume', 'ret_01', 'out_share', 'sic_comp', 'ind_crsp', 'ind_comp',
       '1t24', '12t24', 'acc', 'value_permno', 'value_ind', 'vwr_weight',
       'vwr_permno', 'vwr_ind', 'vol', 'vol_ind_rank', 'vol_value', 'vol_ind',
       'vol_market_rank', 'vol_weight', 'vol_vw', 'vol_vw_change', 'turnover',
       'turnover_ind_rank', 'turnover_value', 'turnover_vw',
       'turnover_market_rank', 'turnover_vw_change', 'first_date', 'age',
       'age_ind_rank', 'age_value', 'age_ind', 'age_market_rank', 'ewr_ind',
       'age_weight', 'awr_permno', 'awr_ind', 'age_tilt', 'adj_share_pct',
       'issued_equity', 'issuance', 'btm', 'btm_weight', 'btm_vwr', 'revenue',
       'revenue_ind_rank', 'revenue_value', 'revenue_ind',
       'revenue_market_rank', 'index', 'cape', 'permno_weighted',
       'market_return', 'two_vwr_ind'],
      dtype='object')

In [399]:
final = ind_data.drop(['share_code', 'sic_crsp', 'ticker', 'nasdaq', 'adj_share', 'close', 'volume', 'ret_01', 'out_share',
                       'sic_comp', 'ind_crsp', 'ind_comp', 'value_ind', 'value_permno', 'value_ind', 'vwr_weight', 
                       'vwr_permno', 'vol_ind_rank', 'vol_value', 'vol_ind', 'vol_market_rank',
                       'turnover_ind_rank', 'turnover_value', 'btm_weight', 'vol_weight', 'btm',
                       'first_date', 'age', 'ewr_ind', 'age_ind_rank', 'age_value', 'age_ind', 'age_weight',
                       'awr_permno', 'awr_ind', 'adj_share_pct', 'issued_equity', 'revenue', 'revenue_ind_rank', 
                       'revenue_value', 'revenue_ind', 'index', '1t24', '12t24', 'vol', 'turnover', 'turnover_market_rank', 'permno_weighted'], axis=1)

In [400]:
final.columns

Index(['acc', 'vwr_ind', 'vol_vw', 'vol_vw_change', 'turnover_vw',
       'turnover_vw_change', 'age_market_rank', 'age_tilt', 'issuance',
       'btm_vwr', 'revenue_market_rank', 'cape', 'market_return',
       'two_vwr_ind'],
      dtype='object')

# Convert to Date/Ind Pair

In [401]:
def convert(data, column):
    collect = []
    for industry, df in data.groupby('ind'):
        df = df[[column]]
        x = df.unstack('permno')
        x.columns = x.columns.get_level_values(0)
        x = x.reset_index().drop('ind', axis=1).set_index('date')
        x[industry] = x.apply(lambda row: row.dropna().iloc[0] if not row.dropna().empty else np.nan, axis=1)
        collect.append(x[[industry]])

    data = pd.concat(collect, axis=1)
    data = data.stack().to_frame()
    data.index.names = ['date', 'ind']
    data = data.rename(columns={0:column})

    return data

In [402]:
collect = []
for col in final.columns:
    print('-----------------------------')
    print(col)
    collect.append(convert(final, col))
collect = pd.concat(collect, axis = 1)

-----------------------------
acc
-----------------------------
vwr_ind
-----------------------------
vol_vw
-----------------------------
vol_vw_change
-----------------------------
turnover_vw
-----------------------------
turnover_vw_change
-----------------------------
age_market_rank
-----------------------------
age_tilt
-----------------------------
issuance
-----------------------------
btm_vwr
-----------------------------
revenue_market_rank
-----------------------------
cape
-----------------------------
market_return
-----------------------------
two_vwr_ind


In [403]:
def create_multi_index(data, stock):
    all_dates = data.index.repeat(len(stock))
    all_stocks = pd.Series(stock * len(data)).values

    multi_index = pd.MultiIndex.from_tuples(list(zip(all_dates, all_stocks)), names=['date', 'ind'])
    df_multi = pd.DataFrame(data.values.repeat(len(stock), axis=0), index=multi_index, columns=data.columns)
    
    return df_multi

def get_stock_idx(data):
    return [stock for stock, df in data.groupby(data.index.names[1], group_keys=False)]

In [404]:
risk = create_multi_index(rf, get_stock_idx(collect))
output = collect.merge(risk, left_index=True, right_index=True, how='left')
output = output.replace([np.inf, -np.inf], np.nan)
output.to_parquet(root / 'data' / 'predictors.parquet.brotli', compression='brotli')